In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from pycaret.classification import *
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score,auc,roc_curve,r2_score,confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.ensemble import RandomForestClassifier
import pickle

In [ ]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

df_new = df.drop(columns =['customerID','TotalCharges'],axis = 1)

df_2 = df_new.copy()

label_encoding_columns = ['gender', 'Partner', 'Dependents', 'PaperlessBilling', 'PhoneService', 'Churn']
for column in label_encoding_columns:
    if column == 'gender':
        df_2[column] = df_2[column].map({'Female': 1, 'Male': 2})
    else: 
        df_2[column] = df_2[column].map({'Yes': 1, 'No': 0}) 
        

one_hot_encoding_columns = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                            'TechSupport', 'StreamingTV',  'StreamingMovies', 'Contract', 'PaymentMethod']

df_2 = pd.get_dummies(df_2, columns = one_hot_encoding_columns, drop_first = True)

In [ ]:
X = df_2.drop(columns='Churn')
y = df_2.loc[:, 'Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=40, shuffle=True, stratify = y)

model = RandomForestClassifier()

clf = model.fit(X_train,y_train)

predictions = clf.predict(X_test)

In [ ]:
#evaluation
precision, recall, f1, _ = score(y_test, predictions)

df_kpi = pd.DataFrame( columns=['KPI Name','KPI Value (0) %','KPI Value (1) %'])

df_kpi = df_kpi.append({'KPI Name': 'Precision', 'KPI Value (0) %': round(precision[0]*100,2),
                           'KPI Value (1) %': round(precision[1]*100,2)}, ignore_index=True)

df_kpi = df_kpi.append({'KPI Name': 'Recall', 'KPI Value (0) %': round(f1[0]*100,2),
                           'KPI Value (1) %': round(recall[1]*100,2)}, ignore_index=True)

df_kpi = df_kpi.append({'KPI Name': 'f1', 'KPI Value (0) %': round(f1[0]*100,2),
                           'KPI Value (1) %': round(recall[1]*100,2)}, ignore_index=True)

 
confusion_matrix1 = pd.crosstab(y_test, predictions, rownames=['Actual'], colnames=['Predicted'])

sns.heatmap(confusion_matrix1, annot=True,fmt="d")

print(df_kpi)

In [ ]:
!pip install interpret-community
!pip install raiwidgets

In [ ]:
from raiwidgets import ErrorAnalysisDashboard
from raiwidgets import ExplanationDashboard
from interpret_community import TabularExplainer
from interpret_community.common.constants import ShapValuesOutput, ModelTask

In [ ]:
explainer = TabularExplainer(clf, 
                             initialization_examples=X_train, 
                             features=X_train.columns, 
                             classes=['0','1'], #binary classification
                             model_task= ModelTask.Classification
                             )

global_explanation = explainer.explain_global(X_test)

ErrorAnalysisDashboard(global_explanation, clf, dataset=X_test, true_y=y_test)